In [1]:
import os, sys
from glob import glob
import numpy as np
import dask
import xarray as xr
import xgcm
import cartopy.crs as ccrs
from cmocean import cm
import cartopy.feature as cfeature

from matplotlib import pyplot as plt
%matplotlib inline

from mitequinox.utils import *
from mitequinox.sigp import *
from mitequinox.plot import *
from dask import compute, delayed

In [2]:
from dask_jobqueue import PBSCluster
# for heavy processing:
cluster = PBSCluster(cores=6, processes=6,  walltime='04:00:00')
w = cluster.scale(12*10)

In [3]:
# get dask handles and check dask server status
from dask.distributed import Client
client = Client(cluster)

In [4]:
client

Client Scheduler: tcp://10.135.36.243:40981 Dashboard: /user/yux/proxy/8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


_____________
# Read data

In [5]:
dij=4
time_length = 240

ds_ice = xr.open_zarr(work_data_dir+'xy/sea_ice_mask.zarr')
ice = ds_ice.AREA.isel(i=slice(0,None,dij), j=slice(0,None,dij))

grd = load_grd().reset_coords().isel(i=slice(0,None,dij), j=slice(0,None,dij))
mask = ((grd.hFacW.rename({'i_g': 'i'}) == 1) &
        (grd.hFacS.rename({'j_g': 'j'}) == 1) 
       ).rename('mask').reset_coords(drop=True)
grd_rspec = xr.merge([grd.XC, grd.YC, grd.Depth, grd.CS, grd.SN])
grd_rspec

<xarray.Dataset>
Dimensions:  (face: 13, i: 1080, j: 1080)
Coordinates:
  * face     (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
  * i        (i) int64 0 4 8 12 16 20 24 ... 4292 4296 4300 4304 4308 4312 4316
  * j        (j) int64 0 4 8 12 16 20 24 ... 4292 4296 4300 4304 4308 4312 4316
Data variables:
    XC       (face, j, i) float32 dask.array<chunksize=(1, 1080, 1080), meta=np.ndarray>
    YC       (face, j, i) float32 dask.array<chunksize=(1, 1080, 1080), meta=np.ndarray>
    Depth    (face, j, i) float32 dask.array<chunksize=(1, 1080, 1080), meta=np.ndarray>
    CS       (face, j, i) float32 dask.array<chunksize=(1, 1080, 1080), meta=np.ndarray>
    SN       (face, j, i) float32 dask.array<chunksize=(1, 1080, 1080), meta=np.ndarray>

In [6]:
# define (real) time
def iters_to_date(iters, delta_t=3600.):
    t0 = datetime.datetime(2011,11,23,8)    
    ltime = delta_t * (np.array(iters))
    dtime = [t0+dateutil.relativedelta.relativedelta(seconds=t) for t in ltime]    
    return dtime

time_day = iters_to_date(np.arange(time_length))

In [7]:
ds = xr.open_zarr(root_data_dir+'zarr/%s.zarr'%('Eta')).isel(time=slice(1512+200,1512+200+time_length))
dsE = ds.isel(i=slice(0,None,dij), j=slice(0,None,dij))
dsE = dsE.assign_coords(time=time_day) 
dsE = dsE.assign_coords(XC=grd_rspec.XC) 
dsE = dsE.assign_coords(YC=grd_rspec.YC) 
print(dsE)
print('\n data size: %.1f GB' %(dsE.nbytes / 1e9))

<xarray.Dataset>
Dimensions:  (face: 13, i: 1080, j: 1080, time: 240)
Coordinates:
    dtime    (time) datetime64[ns] dask.array<chunksize=(240,), meta=np.ndarray>
  * face     (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
  * i        (i) int64 0 4 8 12 16 20 24 ... 4292 4296 4300 4304 4308 4312 4316
    iters    (time) int64 dask.array<chunksize=(1,), meta=np.ndarray>
  * j        (j) int64 0 4 8 12 16 20 24 ... 4292 4296 4300 4304 4308 4312 4316
  * time     (time) datetime64[ns] 2011-11-23T08:00:00 ... 2011-12-03T07:00:00
    XC       (face, j, i) float32 dask.array<chunksize=(1, 1080, 1080), meta=np.ndarray>
    YC       (face, j, i) float32 dask.array<chunksize=(1, 1080, 1080), meta=np.ndarray>
Data variables:
    Eta      (time, face, j, i) float32 dask.array<chunksize=(1, 1, 1080, 1080), meta=np.ndarray>

 data size: 14.7 GB


In [8]:
ds = xr.open_zarr(root_data_dir+'zarr/%s.zarr'%('SSU')).rename({'i_g': 'i'}).isel(time=slice(200,200+time_length))
dsU = ds.isel(i=slice(0,None,dij), j=slice(0,None,dij))
dsU = dsU.assign_coords(time=time_day) 
dsU = dsU.assign_coords(XC=grd_rspec.XC) 
dsU = dsU.assign_coords(YC=grd_rspec.YC) 
print(dsU)
print('\n data size: %.1f GB' %(dsU.nbytes / 1e9))

<xarray.Dataset>
Dimensions:  (face: 13, i: 1080, j: 1080, time: 240)
Coordinates:
    dtime    (time) datetime64[ns] dask.array<chunksize=(240,), meta=np.ndarray>
  * face     (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
  * i        (i) int64 0 4 8 12 16 20 24 ... 4292 4296 4300 4304 4308 4312 4316
    iters    (time) int64 dask.array<chunksize=(1,), meta=np.ndarray>
  * j        (j) int64 0 4 8 12 16 20 24 ... 4292 4296 4300 4304 4308 4312 4316
  * time     (time) datetime64[ns] 2011-11-23T08:00:00 ... 2011-12-03T07:00:00
    XC       (face, j, i) float32 dask.array<chunksize=(1, 1080, 1080), meta=np.ndarray>
    YC       (face, j, i) float32 dask.array<chunksize=(1, 1080, 1080), meta=np.ndarray>
Data variables:
    SSU      (time, face, j, i) float32 dask.array<chunksize=(1, 1, 1080, 1080), meta=np.ndarray>

 data size: 14.7 GB


In [9]:
ds = xr.open_zarr(root_data_dir+'zarr/%s.zarr'%('SSV')).rename({'j_g': 'j'}).isel(time=slice(200,200+time_length))
dsV = ds.isel(i=slice(0,None,dij), j=slice(0,None,dij))
dsV = dsV.assign_coords(time=time_day) 
dsV = dsV.assign_coords(XC=grd_rspec.XC) 
dsV = dsV.assign_coords(YC=grd_rspec.YC) 
print(dsV)
print('\n data size: %.1f GB' %(dsV.nbytes / 1e9))

<xarray.Dataset>
Dimensions:  (face: 13, i: 1080, j: 1080, time: 240)
Coordinates:
    dtime    (time) datetime64[ns] dask.array<chunksize=(240,), meta=np.ndarray>
  * face     (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
  * i        (i) int64 0 4 8 12 16 20 24 ... 4292 4296 4300 4304 4308 4312 4316
    iters    (time) int64 dask.array<chunksize=(1,), meta=np.ndarray>
  * j        (j) int64 0 4 8 12 16 20 24 ... 4292 4296 4300 4304 4308 4312 4316
  * time     (time) datetime64[ns] 2011-11-23T08:00:00 ... 2011-12-03T07:00:00
    XC       (face, j, i) float32 dask.array<chunksize=(1, 1080, 1080), meta=np.ndarray>
    YC       (face, j, i) float32 dask.array<chunksize=(1, 1080, 1080), meta=np.ndarray>
Data variables:
    SSV      (time, face, j, i) float32 dask.array<chunksize=(1, 1, 1080, 1080), meta=np.ndarray>

 data size: 14.7 GB


In [10]:
dsU['u_rotate'] = dsU.SSU*grd_rspec.CS - dsV.SSV*grd_rspec.SN
dsU

<xarray.Dataset>
Dimensions:   (face: 13, i: 1080, j: 1080, time: 240)
Coordinates:
    dtime     (time) datetime64[ns] dask.array<chunksize=(240,), meta=np.ndarray>
  * face      (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
  * i         (i) int64 0 4 8 12 16 20 24 ... 4292 4296 4300 4304 4308 4312 4316
    iters     (time) int64 dask.array<chunksize=(1,), meta=np.ndarray>
  * j         (j) int64 0 4 8 12 16 20 24 ... 4292 4296 4300 4304 4308 4312 4316
  * time      (time) datetime64[ns] 2011-11-23T08:00:00 ... 2011-12-03T07:00:00
    XC        (face, j, i) float32 dask.array<chunksize=(1, 1080, 1080), meta=np.ndarray>
    YC        (face, j, i) float32 dask.array<chunksize=(1, 1080, 1080), meta=np.ndarray>
Data variables:
    SSU       (time, face, j, i) float32 dask.array<chunksize=(1, 1, 1080, 1080), meta=np.ndarray>
    u_rotate  (time, face, j, i) float32 dask.array<chunksize=(1, 1, 1080, 1080), meta=np.ndarray>

In [11]:
dsV['v_rotate'] = dsU.SSU*grd_rspec.SN + dsV.SSV*grd_rspec.CS
dsV

<xarray.Dataset>
Dimensions:   (face: 13, i: 1080, j: 1080, time: 240)
Coordinates:
    dtime     (time) datetime64[ns] dask.array<chunksize=(240,), meta=np.ndarray>
  * face      (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
  * i         (i) int64 0 4 8 12 16 20 24 ... 4292 4296 4300 4304 4308 4312 4316
    iters     (time) int64 dask.array<chunksize=(1,), meta=np.ndarray>
  * j         (j) int64 0 4 8 12 16 20 24 ... 4292 4296 4300 4304 4308 4312 4316
  * time      (time) datetime64[ns] 2011-11-23T08:00:00 ... 2011-12-03T07:00:00
    XC        (face, j, i) float32 dask.array<chunksize=(1, 1080, 1080), meta=np.ndarray>
    YC        (face, j, i) float32 dask.array<chunksize=(1, 1080, 1080), meta=np.ndarray>
Data variables:
    SSV       (time, face, j, i) float32 dask.array<chunksize=(1, 1, 1080, 1080), meta=np.ndarray>
    v_rotate  (time, face, j, i) float32 dask.array<chunksize=(1, 1, 1080, 1080), meta=np.ndarray>

# Movies

In [16]:
font_size = 28

def plot_pretty(v, colorbar=False, title=None, label=None, vmin=None, vmax=None, savefig=None, 
                offline=False, figsize=(20,12), cmmap='balance', ignore_face=[], coast_resolution='110m'):
    
    if vmin is None:
        vmin = v.min()
    if vmax is None:
        vmax = v.max()
    #
    MPL_LOCK = threading.Lock()
    with MPL_LOCK:
        if offline:
            plt.switch_backend('agg')
        #
        fig = plt.figure(figsize=figsize)
        ax = fig.add_subplot(111, projection=ccrs.PlateCarree(central_longitude=0))
        cmap = getattr(cm, cmmap)
        gen = [0,1,2,3,4,5,7,8,9,10,11,12]
        for face in gen:
            vplt = v.sel(face=face)
            if face in [6,7,8,9]:
                # this deals with dateline crossing areas
                im = vplt.where( (vplt.XC<=179.9) & (vplt.XC>=0.)).plot.pcolormesh(ax=ax,                   
                                transform=ccrs.PlateCarree(), vmin=vmin, vmax=vmax,
                                x='XC', y='YC', add_colorbar=colorbar, cmap=cmap)
                im = vplt.where( (vplt.XC>-179.9) & (vplt.XC<=0) ).plot.pcolormesh(ax=ax,                   
                                transform=ccrs.PlateCarree(), vmin=vmin, vmax=vmax,
                                x='XC', y='YC', add_colorbar=colorbar, cmap=cmap)
            else:
                im = vplt.plot.pcolormesh(ax=ax,                   
                                transform=ccrs.PlateCarree(), vmin=vmin, vmax=vmax,
                                x='XC', y='YC', add_colorbar=colorbar, cmap=cmap)        
        
        cb = plt.colorbar(im, ax=ax, ticks=[-4, -3, -2, -1, 0, 1, 2, 3, 4])
        cb.set_label(label=label[0], fontsize=font_size)            
        cb.ax.tick_params(labelsize=font_size)
        ax.set_title(title[0],fontsize=font_size) 
        ax.set_xticks([-180, -135, -90, -45, 0, 45,  90, 135, 180])
        ax.set_xticklabels(['$180\degree$','$135\degree$W','$90\degree$W','$45\degree$W','$0\degree$','$45\degree$E','$90\degree$E','$135\degree$E','$180\degree$'])
        ax.set_ylim(-70., 70.)
        ax.set_yticks([-70, -60, -50, -40, -30, -20, -10, 0, 10, 20, 30, 40, 50, 60, 70])
        ax.set_yticklabels(['','$60\degree$S','','$40\degree$S','','$20\degree$S','','$0\degree$','','$20\degree$N','','$40\degree$N','','$60\degree$N',''])
        ax.tick_params(direction='out', length=6, width=2)
        ax.set_ylabel('',fontsize=font_size)
        ax.set_xlabel('',fontsize=font_size)
        ax.tick_params(labelsize=font_size)
        if coast_resolution is not None:
            ax.coastlines(resolution=coast_resolution, color='k')
        ax.add_feature(cfeature.LAND)    
        
        if savefig is not None:
            fig.savefig(savefig, dpi=150)
            plt.close(fig)
        #
        if not offline:
            plt.show()

# Eta

In [23]:
vmin, vmax = -4, 4
lds = dsE

def genfig(dsE, i, overwrite=True):
    
    # !! passing the variable does not work
    dv = dsE.Eta#.where(ice>0)
    mtime = time_day[i]
    figname = '/home/uz/yux/mit_equinox_backup/hal/Geostrophy_assessment/Figures/Global_Eta/Eta_t%05d'%(i)+'.png'
    #
    if not os.path.isfile(figname) or overwrite:
        #
        #title = 'SSH(m) %s'%mtime
        #label = ''
        title = ['%s'%mtime]
        label = ['$\eta$ (m)', 'u (m/s)', '$u_g$ (m/s)', '$u_a$ (m/s)']
        
        plot_pretty(dv, vmin=vmin, vmax=vmax, cmmap='balance', title=title, label=label, savefig=figname, ignore_face=[6], figsize=(20,6))
        
        m = float(dv.mean().values)
    else:
        m = -1.
    return m

I = range(len(ds['time']))
#I = range(5)
I = [165]
print(I)
values = [delayed(genfig)(lds.isel(time=i), i) for i in I]

[165]


Process all times

In [ ]:
futures = client.compute(values)
%time results = client.gather(futures)

# SSU

In [17]:
vmin, vmax = -2, 2
lds = dsU

def ssufig(dsU, i, overwrite=True):
    
    # !! passing the variable does not work
    dv = dsU.u_rotate#.where(ice>0)
    mtime = time_day[i]
    figname = '/home/uz/yux/mit_equinox_backup/hal/Geostrophy_assessment/Figures/Global_U/SSU_t%05d' %(i)+'.png'
    #
    if not os.path.isfile(figname) or overwrite:
        #
        title = ['%s'%mtime]
        label = ['u (m/s)']
        plot_pretty(dv, vmin=vmin, vmax=vmax, cmmap='balance', title=title, label=label, savefig=figname, ignore_face=[6], figsize=(20,6))

        #m = 1.
        m = float(dv.mean().values)
    else:
        m = -1.
    return m

I = range(len(ds['time']))
#I = range(5)
print(I)
values = [delayed(ssufig)(lds.isel(time=i), i) for i in I]

range(0, 240)


Process all times

In [18]:
futures = client.compute(values)
%time results = client.gather(futures)

CPU times: user 41.1 s, sys: 1.96 s, total: 43.1 s
Wall time: 2min 33s


# SSV

In [25]:
vmin, vmax = -2, 2
lds = dsV

def ssvfig(dsV, i, overwrite=True):
    
    # !! passing the variable does not work
    dv = dsV.v_rotate#.where(ice>0)
    mtime = time_day[i]
    figname = '/home/uz/yux/mit_equinox_backup/hal/Geostrophy_assessment/Figures/Global_V/SSV_t%05d' %(i)+'.png'
    #
    if not os.path.isfile(figname) or overwrite:
        #
        title = ['%s'%mtime]
        label = ['v (m/s)']

        plot_pretty(dv, vmin=vmin, vmax=vmax, cmmap='balance', title=title, label=label, savefig=figname, ignore_face=[6], figsize=(20,6))

        #m = 1.
        m = float(dv.mean().values)
    else:
        m = -1.
    return m

I = range(len(ds['time']))
#I = range(5)
print(I)
values = [delayed(ssvfig)(lds.isel(time=i), i) for i in I]

range(0, 240)


Process all times

In [26]:
futures = client.compute(values)
%time results = client.gather(futures)

CPU times: user 40.7 s, sys: 1.93 s, total: 42.6 s
Wall time: 2min


________________
# Eta, SSU, SSV Together

## surface output from the model

In [27]:
ds = xr.merge([dsE.Eta,dsU.u_rotate,dsV.v_rotate])
ds

<xarray.Dataset>
Dimensions:   (face: 13, i: 1080, j: 1080, time: 240)
Coordinates:
    dtime     (time) datetime64[ns] dask.array<chunksize=(240,), meta=np.ndarray>
  * face      (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
  * i         (i) int64 0 4 8 12 16 20 24 ... 4292 4296 4300 4304 4308 4312 4316
    iters     (time) int64 dask.array<chunksize=(1,), meta=np.ndarray>
  * j         (j) int64 0 4 8 12 16 20 24 ... 4292 4296 4300 4304 4308 4312 4316
  * time      (time) datetime64[ns] 2011-11-23T08:00:00 ... 2011-12-03T07:00:00
    XC        (face, j, i) float32 dask.array<chunksize=(1, 1080, 1080), meta=np.ndarray>
    YC        (face, j, i) float32 dask.array<chunksize=(1, 1080, 1080), meta=np.ndarray>
Data variables:
    Eta       (time, face, j, i) float32 dask.array<chunksize=(1, 1, 1080, 1080), meta=np.ndarray>
    u_rotate  (time, face, j, i) float32 dask.array<chunksize=(1, 1, 1080, 1080), meta=np.ndarray>
    v_rotate  (time, face, j, i) float32 dask.array<chunksize=(1, 1, 1080, 1080), meta=np.ndarray>

In [28]:
font_size = 20

def plot_pretty_3(v1, v2, v3, colorbar=False, title=None, label=None, vmin=None, vmax=None, savefig=None, 
                  offline=False, figsize=(20,12), cmmap='thermal', ignore_face=[], coast_resolution='110m'):
    
    if vmin is None:
        vmin = v.min()
    if vmax is None:
        vmax = v.max()
    #
    MPL_LOCK = threading.Lock()
    with MPL_LOCK:
        if offline:
            plt.switch_backend('agg')
        #
        fig = plt.figure(figsize=figsize)
        cmap = getattr(cm, cmmap)
        
        # 1
        ax = fig.add_subplot(311, projection=ccrs.PlateCarree(central_longitude=0))
        gen = [0,1,2,3,4,5,7,8,9,10,11,12]
        for face in gen:
            vplt = v1.sel(face=face)
            if face in [6,7,8,9]:
                # this deals with dateline crossing areas
                im = vplt.where( (vplt.XC<=179.9) & (vplt.XC>=0.)).plot.pcolormesh(ax=ax,                   
                                transform=ccrs.PlateCarree(), vmin=vmin, vmax=vmax,
                                x='XC', y='YC', add_colorbar=colorbar, cmap=cmap)
                im = vplt.where( (vplt.XC>-179.9) & (vplt.XC<=0) ).plot.pcolormesh(ax=ax,                   
                                transform=ccrs.PlateCarree(), vmin=vmin, vmax=vmax,
                                x='XC', y='YC', add_colorbar=colorbar, cmap=cmap)
            else:
                im = vplt.plot.pcolormesh(ax=ax,                   
                                transform=ccrs.PlateCarree(), vmin=vmin, vmax=vmax,
                                x='XC', y='YC', add_colorbar=colorbar, cmap=cmap)        
        cb = plt.colorbar(im, ax=ax, ticks=[-4, -3, -2, -1, 0, 1, 2, 3, 4])
        cb.set_label(label=label[0], fontsize=font_size)            
        cb.ax.tick_params(labelsize=font_size)
        ax.set_title(title[0],fontsize=font_size) 
        ax.set_xticks([-180, -135, -90, -45, 0, 45,  90, 135, 180])
        ax.set_xticklabels(['$180\degree$W','$135\degree$W','$90\degree$W','$45\degree$W','$0\degree$','$45\degree$E','$90\degree$E','$135\degree$E','$180\degree$E'])
        ax.set_ylim(-70., 70.)
        ax.set_yticks([-60, -50, -40, -30, -20, -10, 0, 10, 20, 30, 40, 50, 60])
        ax.set_yticklabels(['$60\degree$S','','$40\degree$S','','$20\degree$S','','$0\degree$','','$20\degree$N','','$40\degree$N','','$60\degree$N'])
        ax.tick_params(direction='out', length=6, width=2)
        ax.set_ylabel('',fontsize=font_size)
        ax.set_xlabel('',fontsize=font_size)
        ax.tick_params(labelsize=font_size)
        if coast_resolution is not None:
            ax.coastlines(resolution=coast_resolution, color='k')
        ax.add_feature(cfeature.LAND)  

        # 2
        ax = fig.add_subplot(312, projection=ccrs.PlateCarree(central_longitude=0))
        gen = [0,1,2,3,4,5,7,8,9,10,11,12]
        for face in gen:
            vplt = v2.sel(face=face)
            if face in [6,7,8,9]:
                # this deals with dateline crossing areas
                im = vplt.where( (vplt.XC<=179.9) & (vplt.XC>=0.)).plot.pcolormesh(ax=ax,                   
                                transform=ccrs.PlateCarree(), vmin=-2, vmax=2,
                                x='XC', y='YC', add_colorbar=colorbar, cmap=cmap)
                im = vplt.where( (vplt.XC>-179.9) & (vplt.XC<=0) ).plot.pcolormesh(ax=ax,                   
                                transform=ccrs.PlateCarree(), vmin=-2, vmax=2,
                                x='XC', y='YC', add_colorbar=colorbar, cmap=cmap)
            else:
                im = vplt.plot.pcolormesh(ax=ax,                   
                                transform=ccrs.PlateCarree(), vmin=-2, vmax=2,
                                x='XC', y='YC', add_colorbar=colorbar, cmap=cmap)   
        cb = plt.colorbar(im, ax=ax, ticks=[-2, -1, 0, 1, 2])
        cb.set_label(label=label[1], fontsize=font_size)            
        cb.ax.tick_params(labelsize=font_size)
        ax.set_title(title[2],fontsize=font_size) 
        ax.set_xticks([-180, -135, -90, -45, 0, 45,  90, 135, 180])
        ax.set_xticklabels(['$180\degree$W','$135\degree$W','$90\degree$W','$45\degree$W','$0\degree$','$45\degree$E','$90\degree$E','$135\degree$E','$180\degree$E'])
        ax.set_ylim(-70., 70.)
        ax.set_yticks([-60, -50, -40, -30, -20, -10, 0, 10, 20, 30, 40, 50, 60])
        ax.set_yticklabels(['$60\degree$S','','$40\degree$S','','$20\degree$S','','$0\degree$','','$20\degree$N','','$40\degree$N','','$60\degree$N'])
        ax.tick_params(direction='out', length=6, width=2)
        ax.set_ylabel('',fontsize=font_size)
        ax.set_xlabel('',fontsize=font_size)
        ax.tick_params(labelsize=font_size)        
        if coast_resolution is not None:
            ax.coastlines(resolution=coast_resolution, color='k')
        ax.add_feature(cfeature.LAND)  

        # 3
        ax = fig.add_subplot(313, projection=ccrs.PlateCarree(central_longitude=0))
        gen = [0,1,2,3,4,5,7,8,9,10,11,12]
        for face in gen:
            vplt = v3.sel(face=face)
            if face in [6,7,8,9]:
                # this deals with dateline crossing areas
                im = vplt.where( (vplt.XC<=179.9) & (vplt.XC>=0.)).plot.pcolormesh(ax=ax,                   
                                transform=ccrs.PlateCarree(), vmin=-2, vmax=2,
                                x='XC', y='YC', add_colorbar=colorbar, cmap=cmap)
                im = vplt.where( (vplt.XC>-179.9) & (vplt.XC<=0) ).plot.pcolormesh(ax=ax,                   
                                transform=ccrs.PlateCarree(), vmin=-2, vmax=2,
                                x='XC', y='YC', add_colorbar=colorbar, cmap=cmap)
            else:
                im = vplt.plot.pcolormesh(ax=ax,                   
                                transform=ccrs.PlateCarree(), vmin=-2, vmax=2,
                                x='XC', y='YC', add_colorbar=colorbar, cmap=cmap)   
        cb = plt.colorbar(im, ax=ax, ticks=[-2, -1, 0, 1, 2])
        cb.set_label(label=label[2], fontsize=font_size)            
        cb.ax.tick_params(labelsize=font_size)
        ax.set_title(title[2],fontsize=font_size) 
        ax.set_xticks([-180, -135, -90, -45, 0, 45,  90, 135, 180])
        ax.set_xticklabels(['$180\degree$W','$135\degree$W','$90\degree$W','$45\degree$W','$0\degree$','$45\degree$E','$90\degree$E','$135\degree$E','$180\degree$E'])
        ax.set_ylim(-70., 70.)
        ax.set_yticks([-60, -50, -40, -30, -20, -10, 0, 10, 20, 30, 40, 50, 60])
        ax.set_yticklabels(['$60\degree$S','','$40\degree$S','','$20\degree$S','','$0\degree$','','$20\degree$N','','$40\degree$N','','$60\degree$N'])
        ax.tick_params(direction='out', length=6, width=2)
        ax.set_ylabel('',fontsize=font_size)
        ax.set_xlabel('',fontsize=font_size)
        ax.tick_params(labelsize=font_size)                
        if coast_resolution is not None:
            ax.coastlines(resolution=coast_resolution, color='k')
        ax.add_feature(cfeature.LAND)    
        
        if savefig is not None:
            fig.savefig(savefig, dpi=150)
            plt.close(fig)
        #
        if not offline:
            plt.show()

In [31]:
vmin, vmax = -4, 4
lds = ds

def genfig(ds, i, overwrite=True):
    
    # !! passing the variable does not work
    eta = ds.Eta#.where(ice>0)
    u = ds.u_rotate#.where(ice>0)
    v = ds.v_rotate#.where(ice>0)
    
    mtime = time_day[i]
    figname = '/home/uz/yux/mit_equinox_backup/hal/Geostrophy_assessment/Figures/Global_Eta_UV/Eta_UV_t%05d' %(i)+'.png'
    #
    if not os.path.isfile(figname) or overwrite:
        
    
        title = ['%s'%mtime, '', '']
        label = ['$\eta$ (m)', 'u (m/s)', '$v$ (m/s)']
        
        plot_pretty_3(eta, u, v, vmin=vmin, vmax=vmax, cmmap='balance', title=title, label=label, savefig=figname, figsize=(20,18), ignore_face=[6])

        #m = 1.
        m = float(eta.mean().values)
    else:
        m = -1.
    return m

I = range(len(ds['time']))
#I = range(5)
print(I)
values = [delayed(genfig)(lds.isel(time=i), i) for i in I]

range(0, 240)


Process all times

In [ ]:
futures = client.compute(values)
%time results = client.gather(futures)

In [6]:
cluster.close()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError
